In [1]:
! pip install urlextract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import ast
import time
import re
from urlextract import URLExtract as extract
extractor = extract()

import requests as r
import urllib3

In [3]:
# BFE Abnahme
packages = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/package_search?fq=organization:(bundesamt-fur-energie-bfe)&rows=500'

# BFE PROD
#packages = 'https://ckan.opendata.swiss/api/3/action/package_search?fq=organization:(bundesamt-fur-energie-bfe)&rows=500'

# Make the HTTP request
response = requests.get(packages)

# Use the json module to load CKAN's response into a dictionary
response_dict = json.loads(response.content)

# Check the contents of the response
assert response_dict['success'] is True  # make sure if response is OK

# Get a list of all publications and their information (each publication is a dictionary)
data = response_dict['result']['results'].copy()

# get all upper keys
allKeys = []

# store information about each dataset in a frame
df = pd.DataFrame(data)[['id','title','url', 'relations', 'resources']]
df

,id,title,url,relations,resources
0,a02f2157-1cfd-436e-909d-2e197a86f6bd,"{'fr': 'test_relations_01', 'de': 'test_relati...",https://www.bfe.admin.ch/bfe/de/home.html,"[{'url': 'http://de.wikipedia.org/wiki/Pizza',...","[{'byte_size': 0, 'coverage': '', 'cache_last_..."
1,02512543-1db1-4992-9aa7-b8eaf90069ab,"{'fr': '', 'de': 'Erdbebengefährdung für die S...",https://www.bfe.admin.ch/bfe/de/home/versorgun...,"[{'url': 'http://doi.org/10.5169/seals-73629',...","[{'byte_size': 0, 'coverage': '', 'cache_last_..."
2,44f9dd93-6bd0-4d56-8650-e472b44b72b0,{'fr': 'Offres d’encouragement de l’innovation...,http://www.bfe.admin.ch/ogd4,{},"[{'byte_size': 0, 'coverage': '', 'cache_last_..."
3,b4117aac-fc84-43de-8c0f-a74e0dd98247,"{'fr': '', 'de': '', 'en': 'test_relations', '...",https://www.bfe.admin.ch,[[{'url': 'https://www.bfe.admin.ch/bfe/de/hom...,"[{'byte_size': 0, 'coverage': '', 'cache_last_..."
4,75b319b5-a7bc-40fc-80e9-579b02e46c02,"{'fr': '', 'de': 'Kennzahlen Shared Mobility',...",https://bfe.admin.ch/shared-mobility-angebote,[[{'url': 'https://www.sharedmobility.ch/info'...,"[{'byte_size': 0, 'cache_last_updated': None, ..."
...,...,...,...,...,...
131,ff382622-8419-4b66-a157-1e47b9653883,{'fr': 'Chiffres-clés concernant l’infrastruct...,http://www.uvek-gis.admin.ch/BFE/storymaps/MO_...,{},"[{'byte_size': 0, 'cache_last_updated': None, ..."
132,a37cb9ae-50f9-4fa5-a48c-95e2c5f0625b,{'fr': 'Aptitude des toitures à utiliser l’éne...,https://www.bfe.admin.ch/bfe/de/home/versorgun...,{},"[{'byte_size': 0, 'cache_last_updated': None, ..."
133,73d41982-f474-409a-a129-418678921128,{'fr': 'Etude sur l'électricité hivernale d'or...,https://www.bfe.admin.ch/bfe/de/home/politik/e...,[{'url': 'https://www.bfe.admin.ch/bfe/de/home...,"[{'byte_size': 5819680686, 'cache_last_updated..."
134,e839fa78-ba22-4772-be16-0843b99b7161,{'fr': 'Résultats de la modélisation PE2050+ :...,https://www.bfe.admin.ch/bfe/de/home/politik/e...,[{'url': 'https://www.uvek-gis.admin.ch/BFE/st...,"[{'byte_size': 0, 'cache_last_updated': None, ..."


In [4]:
print(df['relations'].loc[2])
print(type(df['relations'].loc[2]))

{}
<class 'dict'>


In [5]:
print(df['relations'].loc[4])
print(type(df['relations'].loc[4]))

["[{'url': 'https://www.sharedmobility.ch/info', 'label': 'Dokumentation von sharedmobility.ch'}]"]
<class 'list'>


In [6]:
print(df['relations'].loc[0])
print(type(df['relations'].loc[0]))

[{'url': 'http://de.wikipedia.org/wiki/Pizza', 'label': 'relation1'}, {'url': 'www.wikipedia.org/wiki/Margarethe_von_Italien', 'label': 'relation2'}]
<class 'list'>


--------------------------------------------------------------------------------

[{'url': 'https://de.wikipedia.org/wiki/Pizza', 'label': 'relation1'}, {'url': 'https://en.wikipedia.org/wiki/Main_Page', 'label': 'relation2'}]

In [8]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

s = r.Session()

proxies = None

private_api_key = 'cc1c806e-1fd9-4180-a6bf-7a5304ca503a'

patch_url = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/package_patch'

header = {'Authorization': private_api_key}

In [9]:
df.iloc[0]

id                        a02f2157-1cfd-436e-909d-2e197a86f6bd
title        {'fr': 'test_relations_01', 'de': 'test_relati...
url                  https://www.bfe.admin.ch/bfe/de/home.html
relations    [{'url': 'http://de.wikipedia.org/wiki/Pizza',...
resources    [{'byte_size': 0, 'coverage': '', 'cache_last_...
Name: 0, dtype: object

In [10]:
data = {'id': 'a02f2157-1cfd-436e-909d-2e197a86f6bd',
        'relations': [{'url': 'https://de.wikipedia.org/wiki/Pizza', 'label': 'relation1'}, {'url': 'https://en.wikipedia.org/wiki/Main_Page', 'label': 'relation2'}]}
    
print(data)
    
x = s.post(patch_url,data=data,proxies=proxies,headers=header,verify=False)
print(x.status_code)

{'id': 'a02f2157-1cfd-436e-909d-2e197a86f6bd', 'relations': [{'url': 'https://de.wikipedia.org/wiki/Pizza', 'label': 'relation1'}, {'url': 'https://en.wikipedia.org/wiki/Main_Page', 'label': 'relation2'}]}
200
